### 잠재 요인 협업 필터링
- 사용자-아이템 평점 matrix 속에 숨어 있는 잠재 요인을 추출하여 추천 
- 사용자-아이템 간의 상호작용은, 사용자 벡터와 아이템 벡터의 내적으로 정의됨
![matrix](./matrix.png)

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [ ]:
# 위를 바탕으로 모델 정의하기
class GMF(nn.Module):
    # def __init__(self) :
    def __init__(self, user_num, item_num, factor_num, num_layers, dropout, model,):
        super(GMF, self).__init__()
        self.dropout = dropout
        self.model = model
        
        # 임베딩 저장공간 확보 : num_embeddings, embedding_dim
        self.embed_user_GMF = nn.Embedding(user_num, factor_num)
        self.embed_item_GMF = nn.Embedding(item_num, factor_num)
        predict_size = factor_num
        
        self.predict_layer = nn.Linear(predict_size, 1)
        self._init_weight_()
        
    def _init_weight_(self):
        # weight 초기화
        nn.init.normal_(self.embed_user_GMF.weight, std=0.01)
        nn.init.normal_(self.embed_item_GMF.weight, std=0.01)
        nn.init.kaiming_uniform_(self.predict_layer.weight, a=1, nonlinearity="sigmoid")
        
        # bias 초기화
        for m in self.modules():
            if isinstance(m, nn.Linear) and m.bias is not None:
                m.bias.data.zero_()
    
    def forward(self, user, item):
        embed_user_GMF = self.embed_user_GMF(user)
        embed_item_GMF = self.embed_item_GMF(item)
        output_GMF = embed_user_GMF * embed_item_GMF
        concat = output_GMF
        
        prediction = self.predict_layer(concat)
        return prediction.view(-1)
        

- nn.Embedding : 정수나 단어를 one hot 으로 encoding하면 대부분이 0인(sparse) 공간낭비. 이거를 연속적인 실수로 표현하게 하는 역할
    - (임베딩할 단어 수, 임베딩할 차원)
    - 결과는 .weight 하면 할당된 layer를 볼 수 있음
- nn.init.normal : layer의 값들을 다 초기화
- nn.Linear(x_dim, y_dim) : 입력된 x차원에 따라 y차원으로 선형회귀
- view()는 tensor 행렬의 shape을 바꿔주는 역할임